In [ ]:
!pip install -r requirements.txt

In [3]:
import pandas as pd
from neuralprophet import NeuralProphet

In [6]:
#No estamos importando kansas2019 porque esa información es de Kansas City, actualmente la data está siendo extraída para
#comprobar los modelos de predicción
#kansas2019 = pd.read_csv('./DATA/kansas2019-2022.csv')[columns]

kansas2016 = pd.read_csv('./DATA/kansas2016-2019.csv')
kansas2013 = pd.read_csv('./DATA/kansas2013-2016.csv')

In [7]:
kansas2013.columns

Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')

In [10]:
columns = ["datetime", "temp"]

In [11]:
df2016, df2013 = kansas2016[columns], kansas2013[columns]
df2016

,datetime,temp
0,2016-07-14,24.5
1,2016-07-15,23.9
2,2016-07-16,25.8
3,2016-07-17,29.0
4,2016-07-18,28.5
...,...,...
994,2019-04-04,13.3
995,2019-04-05,13.0
996,2019-04-06,16.7
997,2019-04-07,17.5


In [15]:
all_kansas = pd.concat([df2016, df2013]).sort_values('datetime').reset_index().drop(columns=['index']).rename(columns={'datetime':'ds', 'temp':'y'})
all_kansas

,ds,y
0,2013-10-19,7.9
1,2013-10-20,12.9
2,2013-10-21,9.7
3,2013-10-22,10.8
4,2013-10-23,6.2
...,...,...
1993,2019-04-04,13.3
1994,2019-04-05,13.0
1995,2019-04-06,16.7
1996,2019-04-07,17.5


In [16]:
#Checando que la data no tenga valores NaN (Not a Number)
all_kansas[all_kansas.isna().any(axis=1)]


,ds,y


In [18]:
train_dataset = all_kansas.iloc[:1500, :]
test_dataset = all_kansas.iloc[1500:, :]
test_dataset

,ds,y
1500,2017-11-27,13.4
1501,2017-11-28,13.4
1502,2017-11-29,9.6
1503,2017-11-30,6.6
1504,2017-12-01,5.3
...,...,...
1993,2019-04-04,13.3
1994,2019-04-05,13.0
1995,2019-04-06,16.7
1996,2019-04-07,17.5


In [23]:
kansas_model = NeuralProphet(
    n_forecasts=2,
    n_lags=5,
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False,
    epochs=500
)

In [24]:
kansas_model.fit(train_dataset, freq='D')

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.933% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32


  0%|          | 0/130 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.18E-02, min: 1.06E+01


  0%|          | 0/130 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 6.18E-02, min: 4.73E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 7.25E-02
Epoch[500/500]: 100%|███████████| 500/500 [00:44<00:00, 11.25it/s, SmoothL1Loss=0.00419, MAE=3.25, RMSE=4.23, RegLoss=0]


,SmoothL1Loss,MAE,RMSE,RegLoss
0,2.390977,134.430451,136.165314,0.0
1,2.204535,125.760895,127.390555,0.0
2,2.012283,116.821166,118.369650,0.0
3,1.810111,107.420152,108.921266,0.0
4,1.594495,97.394027,98.881310,0.0
...,...,...,...,...
495,0.004194,3.250727,4.222417,0.0
496,0.004193,3.250305,4.232124,0.0
497,0.004193,3.250383,4.227208,0.0
498,0.004193,3.250081,4.229565,0.0


In [27]:
forecast = kansas_model.predict(test_dataset)
forecast.iloc[5:,:]

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.799% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
WARNING - (py.warnings._showwarnmsg) - /mnt/c/TEC/OCTAVO_b/METODOS/env_jpy/lib/python3.8/site-packages/neuralprophet/forecaster.py:2098: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(future_df)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.8% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
WARNING - (py.warnings._showwarnmsg) - /mnt/c/TEC/OCTAVO_b/METODOS/env_jpy/lib/python3.8/site-packages/neuralprophet/forecaster.py:1406: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_end_to_append)



,ds,y,yhat1,residual1,yhat2,residual2,ar1,ar2,trend
5,2017-12-02,6.4,8.163616,1.763616,None,NaN,20.851122,None,-12.687505
6,2017-12-03,10.3,9.172272,-1.127728,9.46244,-0.83756,21.85742,22.147587,-12.685148
7,2017-12-04,14.5,11.868807,-2.631193,9.748121,-4.751879,24.551598,22.430914,-12.682793
8,2017-12-05,2.8,14.453556,11.653556,10.795368,7.995368,27.133993,23.475803,-12.680435
9,2017-12-06,3.5,3.175617,-0.324383,12.413258,8.913258,15.853696,25.091337,-12.67808
...,...,...,...,...,...,...,...,...,...
493,2019-04-04,13.3,15.520126,2.220126,10.20224,-3.09776,27.057701,21.739817,-11.537575
494,2019-04-05,13.0,13.316893,0.316893,12.76977,-0.23023,24.852114,24.304991,-11.535219
495,2019-04-06,16.7,14.912369,-1.787631,12.126106,-4.573894,26.445232,23.658968,-11.532862
496,2019-04-07,17.5,17.939323,0.439323,13.961338,-3.538662,29.469831,25.491844,-11.530506
